In [1]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier, BaggingClassifier
from sklearn.multioutput import MultiOutputRegressor
import sklearn
from random import random
from sklearn.model_selection import cross_validate
import matplotlib
from waveform_methods import binning
import pandas as pd

In [2]:
file = uproot.open('/home/andy/12360.root')
file2 = uproot.open('/home/andy/12362.root')

In [3]:
print(file['LaputopParams'].keys())

[b'Run', b'Event', b'SubEvent', b'SubEventStream', b'SubEventStream_String', b'exists', b's125', b'beta', b'age', b'A', b'D', b'N', b'xc_err', b'yc_err', b'ny_err', b'nx_err', b'tc_err', b'log10_s125_err', b'beta_err', b'A_err', b'D_err', b'N_err', b's50', b's70', b's80', b's100', b's150', b's180', b's250', b's500', b'e_proton', b'e_iron', b'e_h4a', b'llh', b'llh_silent', b'chi2', b'chi2_time', b'ndf', b'rlogl', b'nmini']


In [4]:
Xmax1 = file['MCPrimaryInfo']['ghMaxDepth'].array()
valueDepth = file['MCPrimaryInfo']['longNumCharged'].array()
Depth1 = file['MCPrimaryInfo']['longDepth'].array()
chi2_1 = file['CurvatureOnlyParams']['chi2_time'].array()
red1 = file['MCPrimaryInfo']['ghRedChiSqr'].array()
mass1 = [1 for i in range(len(red1))]

good = []
count = 0
for i in range(len(Xmax1)):
    if (Xmax1[i] > 400 and Xmax1[i]<900) and red1[i] < 300:
        good.append(1)
        count+=1
    else:
        good.append(0)
        
Xmax2 = file2['MCPrimaryInfo']['ghMaxDepth'].array()
valueDepth = file2['MCPrimaryInfo']['longNumCharged'].array()
Depth2 = file2['MCPrimaryInfo']['longDepth'].array()
chi2_2 = file2['CurvatureOnlyParams']['chi2_time'].array()
red2 = file2['MCPrimaryInfo']['ghRedChiSqr'].array()
mass2 = [4 for i in range(len(red2))]

good2 = []
for i in range(len(Xmax2)):
    if (Xmax2[i] > 350 and Xmax2[i] < 650) and red2[i] < 300:
        good2.append(1)
        count+=1
    else:
        good2.append(0)

In [5]:
S125_1 = np.log10(file['LaputopParams']['s125'].array())
S125_2 = np.log10(file2['LaputopParams']['s125'].array())
A1 = file['CurvatureOnlyParams']['A'].array()
A2 = file2['CurvatureOnlyParams']['A'].array()
D1 = file['CurvatureOnlyParams']['D'].array()
D2 = file2['CurvatureOnlyParams']['D'].array()
N1 = file['CurvatureOnlyParams']['N'].array()
N2 = file2['CurvatureOnlyParams']['N'].array()
beta1 = file['LaputopParams']['beta'].array()
beta2 = file2['LaputopParams']['beta'].array()
zenith1 = file['Laputop']['zenith'].array()
zenith2 = file2['Laputop']['zenith'].array()
energy1 = file['MCPrimary']['energy'].array()
energy2 = file2['MCPrimary']['energy'].array()

slc_charge1 = file['LaputopSLCPE']['charge'].array()
slc_charge2 = file2['LaputopSLCPE']['charge'].array()

slc_charge1 = [np.sum(i) for i in slc_charge1]
slc_charge2 = [np.sum(i) for i in slc_charge2]

hlc_charge1 = file['LaputopHLCPE']['charge'].array()
hlc_charge2 = file2['LaputopHLCPE']['charge'].array()

hlc_charge1 = [np.sum(i) for i in hlc_charge1]
hlc_charge2 = [np.sum(i) for i in hlc_charge2]

Qtot1 = [i+j for i,j in zip(slc_charge1,hlc_charge1)]
Qtot2 = [i+j for i,j in zip(slc_charge2,hlc_charge2)]


Xmax1_new = Xmax1
Xmax2_new = Xmax2

In [6]:
A = np.append(A1,A2)
D = np.append(D1,D2)
N = np.append(N1,N2)
S125 = np.append(S125_1,S125_2)
beta = np.log10(np.append(beta1,beta2))
chi2 = np.log10(np.append(chi2_1,chi2_2))
red = np.append(red1,red2)
zenith = np.cos(np.append(zenith1,zenith2))
Xmax = np.append(Xmax1_new,Xmax2_new)
goodness = np.append(good,good2)
red = np.append(red1,red2)
mass = np.append(mass1,mass2)
slc_charge = np.append(slc_charge1,slc_charge2)
hlc_charge = np.append(hlc_charge1,hlc_charge2)
qtot = np.append(Qtot1,Qtot2)

hlc_charge = np.log10(hlc_charge)
qtot = np.log10(qtot)

energy = np.log10(np.append(energy1,energy2))


In [7]:
mask_convert = []
for i in goodness:
    if i == 1:
        mask_convert.append(True)
    else:
        mask_convert.append(False)

In [8]:
print(np.corrcoef(np.array([mass[mask_convert],
                            Xmax[mask_convert],
                            A[mask_convert],
                            D[mask_convert],
                            N[mask_convert],
                            S125[mask_convert],
                            beta[mask_convert],
                            zenith[mask_convert],
                            chi2[mask_convert],
                            red[mask_convert],
                            slc_charge[mask_convert],
                            hlc_charge[mask_convert],
                            qtot[mask_convert]
                            ])))

[[ 1.         -0.64601265 -0.11363941 -0.16931619  0.08145542  0.06441446
  -0.22763337 -0.04440126  0.03758456 -0.09058544  0.32913523  0.02326574
   0.03535179]
 [-0.64601265  1.          0.08745356 -0.01605469 -0.04374051  0.27249902
   0.19097071 -0.13217884 -0.02699269  0.23491313 -0.01841073  0.27416859
   0.26971283]
 [-0.11363941  0.08745356  1.          0.3092822  -0.05572063 -0.03832655
   0.06570221  0.16972394  0.79176294 -0.06808267 -0.12373802 -0.05951345
  -0.06296898]
 [-0.16931619 -0.01605469  0.3092822   1.         -0.04063629 -0.52411156
   0.04819148 -0.01229072  0.18240478 -0.13975348 -0.30926039 -0.4832547
  -0.48386795]
 [ 0.08145542 -0.04374051 -0.05572063 -0.04063629  1.          0.05788232
  -0.05665567 -0.08549465 -0.04557859  0.04619319  0.06619701  0.05100963
   0.05636533]
 [ 0.06441446  0.27249902 -0.03832655 -0.52411156  0.05788232  1.
   0.0745446   0.21541602 -0.11170028  0.23444817  0.37748237  0.83089723
   0.83195647]
 [-0.22763337  0.19097071  0.06

In [40]:
input_variable2_old = np.array([(np.append(np.append(i,j),k),l) for i,j,k,l in zip(A[mask_convert],D[mask_convert],beta[mask_convert],slc_charge[mask_convert])])
output_new_old = np.array(mass[mask_convert])

import random
random.seed(1)

validation = []
output = []

for i in range(len(input_variable2_old)):
    if random.random() < 0.1:
        validation.append(i)
    else:
        output.append(i)
        
validation_x = input_variable2_old[validation]
validation_y = output_new_old[validation]
input_variable = input_variable2_old[output]
output_new = output_new_old[output]

In [41]:
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.2
X_train, X_test, y_train_1, y_test_1 = train_test_split(input_variable,output_new , test_size=test_size, random_state=seed)

In [42]:
weights = np.array(Xmax[mask_convert][output]/np.sum(Xmax[mask_convert][output]))

In [43]:
from scipy.stats import norm
from sklearn.base import clone
import pickle
from statistics import mode

#100,26
for j in range(1,40):
    regr_2 = None
    rng = np.random.RandomState(1)
    for i in range(1,120):
        if regr_2 is None:
            regr_2 = BaggingClassifier(DecisionTreeClassifier(max_depth=j,splitter='random',class_weight='balanced'),
                                        n_estimators=i, random_state=rng, warm_start= True)
        else:
            regr_2.set_params(n_estimators=i)
        regr_2.fit(input_variable,output_new,sample_weight=weights)
    proton = []
    iron = []
    predicted = regr_2.predict(validation_x)
    cm = confusion_matrix(validation_y,predicted)
    proton.append(cm[0][0]/np.sum(cm[0]))
    iron.append(cm[1][1]/np.sum(cm[1]))
    print(np.mean(proton),np.mean(iron),i,j)

ValueError: setting an array element with a sequence.

In [ ]:
#from sklearn.metrics import confusion_matrix

In [ ]:
predict1 = regr_2.predict(validation_x)
    
mass_predict = []
for i in predict1:
    if i <2.5:
        mass_predict.append(1)
    else:
        mass_predict.append(4)
cm=confusion_matrix(validation_y,mass_predict)

print(cm[0][0]/np.sum(cm[0]),cm[1][1]/np.sum(cm[1]))

In [ ]:
import pickle
pickle.dump(regr_1,open('cut_model.sav','wb'))
pickle.dump(regr_2,open('model.sav','wb'))